In [1]:
from supabase import create_client, Client
import os

model_name = "text-embedding-3-small"
openai_api_key = os.environ['OPENAI_API_KEY']
supabase_url = os.environ['SUPABASE_URL']
supabase_api_key = os.environ['SUPABASE_API_KEY'] 
supabase: Client = create_client(supabase_url, supabase_api_key)

In [2]:
from supabase import create_client
from openai import OpenAI
from typing import List, Dict
import numpy as np
# 24比回答：
# 現在我要做的事情是一口氣搞1000題然後去計算時間和用量，開始時間：2040
class DataMigration:
    def __init__(self, supabase_url: str, supabase_key: str, openai_api_key: str):
        self.supabase = create_client(supabase_url, supabase_key)
        self.openai_client = OpenAI(api_key=openai_api_key)


    def process_existing_questions(self, batch_size: int = 10):
        """處理現有的問題並填充新的列"""
        # 獲取所有需要處理的問題
        query = self.supabase.table('multidoc_questions')\
            .select('id', 'question', 'question_embedding')\
            .is_('gpt_answer', None)\
            .execute()
        
        questions = query.data
        total = len(questions)
        print(f"找到 {total} 個需要處理的問題")
        errorcount = 0
        for i in range(0, total, batch_size):
            batch = questions[i:i + batch_size]
            for question in batch:
                try:
                    # 找到相似文檔
                    similar_docs = self.supabase.rpc(
                        'match_documents',
                        {
                            'query_embedding': question['question_embedding'],
                            'match_count': 5
                        }
                    ).execute()

                    # 生成 GPT 答案
                    context = "\n\n".join([
                        f"文章標題: {doc['title']}\n內容: {doc['content']}"
                        for doc in similar_docs.data
                    ])

                    prompt = f"""基於以下文章內容回答問題：

                    文章內容：
                    {context}

                    問題：{question['question']}

                    請提供準確且完整的回答。如果文章內容無法妥善回答問題，請回答「我不知道」"""

                    response = self.openai_client.chat.completions.create(
                        model="gpt-4o",
                        messages=[
                            {"role": "system", "content": "你是一個專業的問答助手。"},
                            {"role": "user", "content": prompt}
                        ]
                    )

                    gpt_answer = response.choices[0].message.content
                    similar_doc_ids = [doc['id'] for doc in similar_docs.data]

                    # 更新數據庫
                    self.supabase.table('multidoc_questions')\
                        .update({
                            'gpt_answer': gpt_answer,
                            'similar_doc_ids': similar_doc_ids
                        })\
                        .eq('id', question['id'])\
                        .execute()

                    print(f"成功處理問題 ID: {question['id']}")
                    print(gpt_answer)

                except Exception as e:
                    print(f"處理問題 ID: {question['id']} 時發生錯誤: {str(e)}")
                    errorcount += 1
                    continue

            print(f"已處理 {min(i + batch_size, total)}/{total} 個問題")
            print(errorcount)

In [3]:
import os
openai_api_key = os.environ['OPENAI_API_KEY']
supabase_url = os.environ['SUPABASE_URL']
supabase_api_key = os.environ['SUPABASE_API_KEY'] 

# 35.47, 510.5

找到 1045 個需要處理的問題
成功處理問題 ID: 25
趙溫聽說大人不見後，心裏如同一塊石頭落地，是因爲這意味著他可以放心，再沒有顧慮了。這在清·李寶嘉《官場現形記》第二回中描述，趙溫因此心裡放下了擔心，趕忙去無慮地完成工作。
成功處理問題 ID: 26
網絡小說《一夜後婚》的作者是下弦の咒。
成功處理問題 ID: 27
《一字記之曰日》是由淡定的錯錯創作的。
成功處理問題 ID: 28
成語“一寸丹心”的出處是唐代杜甫的《鄭附馬池臺喜遇鄭廣文同飲》詩。
成功處理問題 ID: 29
一尾概率指的是計算隨機變量 \( x \) 小於 \( \mu - k\sigma \) 或大於 \( \mu + k\sigma \) 的概率，這被稱為一尾概率（一側概率），通常記作 \( \alpha/2 \)。
成功處理問題 ID: 30
該小說的作者是米問問。
成功處理問題 ID: 31
一座燈塔的光照亮了一條沙灘、一行走過的足跡和一個方向。
成功處理問題 ID: 32
一心亭餐館的特色服務包括提供在線菜單。
成功處理問題 ID: 33
建設單位委託第三方中介機構具體實施分戶驗收工作時，第三方中介機構需要具備相應檢測能力，以負責實施實物測量和功能性檢測的工作。該機構應具備承擔分戶驗收相應責任的能力，並按照合同約定協助建設單位處理日後可能出現的用戶質量投訴工作。
成功處理問題 ID: 34
《一抹綠》這部網絡小說的作者是大綠茶。
已處理 10/1045 個問題
0
成功處理問題 ID: 35
一攬子個稅改革方案是個稅領域的一項深層次改革，要求綜合考慮納稅人的各項收入和支出，綜合計算納稅金額，甚至考慮納稅人家庭負擔等因素，實行基本生計扣除加專項扣除。這將會取代目前實行的按不同來源分類並對每一類按單獨稅率計算的分類所得稅制。這種改革旨在解決深層次的扭曲問題，減少低收入者的稅負，更好地實現稅收調節差距和保障公平的功能。然而，實施分類和綜合相結合的所得稅制在徵管技術和管理體制方面存在不少困難。
成功處理問題 ID: 36
《一日一字》適合於任何漢語水平、任何年齡的對漢字感興趣的學習者。
成功處理問題 ID: 37
我不知道。
成功處理問題 ID: 38
《一本無限流小說》的作者是蒼狼雪月。
成功處理問題 ID: 39
我不知道。根據提供的文章內容，《一條繩倆螞蚱》是由高信演唱的，而不

KeyboardInterrupt: 

成功處理問題 ID: 532
長沙人蔘米是通過以下方式製作的：首先，將大米和糖精放入一個黑色的圓筒中，然後密封好圓筒。在火上烤加熱大米，過程中可以隱約聽到砰砰的聲響。製作完成時，將圓筒的蓋子一端套入一個大口袋裏，猛地敲擊圓筒蓋子上的機關。此時蓋子會打開，圓筒內的高壓環境會發出“嘣”的一聲巨響，人蔘米便做好了。這種製作過程與爆米花相似，但使用的是大米而非玉米粒。
已處理 510/1045 個問題

510問題 35分鐘
525(?) 5020
今天用量  525個900811 tokens

In [4]:
migration = DataMigration(
    supabase_url=supabase_url,
    supabase_key=supabase_api_key,
    openai_api_key=openai_api_key)

# 處理現有數據
migration.process_existing_questions()

# 535個問題搞了33分鐘，用量：

找到 535 個需要處理的問題
成功處理問題 ID: 533
《人口學導論》這本書的編寫目的在於使讀者瞭解人口學的基本體系和主要內容、掌握和運用人口學的基本理論和研究方法，認識人口發展的基本規律，分析和研究現實的各種人口現象和人口問題，以及與人口相關的各種社會經濟問題，並為此打下堅實的理論和方法論基礎。
成功處理問題 ID: 534
人員推廣主要指的是和終端密切相關的促銷人員所做的推廣活動。這是一種原始但有時非常有效的產品促銷策略，通過具體銷售人員直接與消費者互動，推動銷售。人員推廣的優點包括能夠通過現身說法贏得消費者的信任和好感，促成購買行為；然而，它的單位成本較高，管理較困難，且對消費者的覆蓋面不廣。促銷層級的人員包括促銷員、促銷主管、市場代表、市場督導等，這些人不一定是公司正式編制的員工，其工作費用在推廣費用中列支。做好人員推廣的關鍵在於促銷員對產品知識和賣點的掌握，以及其促銷技能。
成功處理問題 ID: 536
仿真人頭錄音方式的原理是把兩個微型全方向性話筒安置在一個與真人頭幾乎一模一樣的假人頭的耳道內，模擬人耳聽到聲音的整個過程。這個假人頭被設計得與真人的耳道、耳廓、頭蓋骨等特徵相似，甚至使用與人體接近的材料製造，以儘可能真實地模擬人耳受聲音影響的過程。這樣錄製的雙聲道信號可以在耳機中重放，幾乎完美地還原錄音場所的360度音場效果。
成功處理問題 ID: 537
進行人崗匹配分析需要從兩個方面進行：人員分析和崗位分析。

1. **人員分析**：這涉及對個人性格和能力的評估，以確定他是否符合崗位要求。
   - **性格分析**：評估個人的性格特徵，如沉着內斂、善言辭、外向等，以及這些特徵是否適合該崗位的工作性質。不同氣質類型（如膽汁質、多血質、粘液質、抑鬱質）對應不同的工作特徵。
   - **能力分析**：評估個人能力，如反應能力、語言表達能力、溝通能力等，以確定其是否適合工作崗位。

2. **崗位分析**：這包括對崗位職責和要求的詳細分析。
   - **崗位職責**：分析該崗位的工作內容和性質。
   - **崗位要求**：分析該崗位對員工所需能力和素質的具體要求。

人崗匹配的宗旨是找到與崗位要求最契合的員工，就像選擇戀人一樣，應選擇最合適的。
成功處理問題 ID: 538
人工材料是指自然界以化合物形式存在的物質，不能直接使用的，或者自然界不

20200958 不知道為什麼都沒有顯示扣錢但是有顯示ｔｏｋｅｎ一直在吃
gpt-4o-2024-08-06
$2.50 / 1M input tokens
$1.25 / 1M input tokens
$1.25 / 1M cached** input tokens
$10.00 / 1M output tokens
用這裡的費用來計算好了，看看有沒有做到類似的內容

12/20 結果 我今天才開始大量用gpt所以之前是別人用的
今天總共呼叫次數應該是1070-24次
用量521086+987341+223592token
輸出33342+66554+5203 token
我12月打出1130request但是這裡只有1070斟酌計算